In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis as maha
import statsmodels.api as sm

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Data Cleaning

- Load Data
- Calculate yield curve slope and Y/Y changes in relevant columns


#### Data Definitions
- TNX: US 10y Treasury
- US_Corp: ML US Corporate Bond Total Return Index
- LIBOR: 3m LIBOR Rate
- BAA: Moody's long-term corporate bond yields index
- UNRATE: US seasonally-adjusted unemployment rate
- SPY: S\&P 500 Index
- IRX: US 3m treasury rate
- RGDP: US seasonally-adjusted Real GDP


In [27]:
#Missing Commodity Index Data
data = pd.read_csv('data/data.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [28]:
#Linear Interpolation Forward fill - for GDP data which is quarterly
#Could potentially want to only use quarterly data - set lin_interp = False
lin_interp = True

if lin_interp:
    data['RGDP'] = data['RGDP'].interpolate()
else:
    data = data.dropna(subset = ['RGDP'])

In [29]:
#Yield Curve Slope: 10y yields - 3m yields
data['YC_Slope'] =  pd.eval('data.TNX - data.IRX')
#Credit Spread: long-term BAA (corp bonds) - 10y treasury rate
data['Cred_Spread'] = pd.eval('data.BAA - data.TNX')

In [30]:
#Fill in Y/Y changes
YY_cols = ['CPI', 'RGDP']
data[[x + '_Growth' for x in YY_cols]] = data[YY_cols]/data[YY_cols].shift(12 if lin_interp else 4) - 1

#MISSING COMMODITIES INDEX AND INTEREST RATES INDEX
assets = ['SPY', 'US_Corp']
data[[x + '_Return' for x in assets]] = data[assets]/data[assets].shift(1) - 1

assets.append('TreasInd')
data['TreasInd_Return'] = data['TreasInd']
assets.append('Cash')
data['Cash_Return'] = data['Cash']

data = data.drop(columns = ['TreasInd' , 'Cash'])

#Drop null rows
data = data.dropna()

#Subtract mean
# data = data - data.mean()

In [31]:
data.median()

TNX                    3.158000
US_Corp             1969.855909
LIBOR                  1.262410
CPI                  221.194000
BAA                    6.020000
UNRATE                 5.500000
SPY                  133.820007
IRX                    0.932000
RGDP               15725.333000
YC_Slope               2.041000
Cred_Spread            2.615000
CPI_Growth             0.020346
RGDP_Growth            0.021447
SPY_Return             0.009834
US_Corp_Return         0.005527
TreasInd_Return        0.000048
Cash_Return            0.000043
dtype: float64

In [32]:
data.head()

,TNX,US_Corp,LIBOR,CPI,BAA,UNRATE,SPY,IRX,RGDP,YC_Slope,Cred_Spread,CPI_Growth,RGDP_Growth,SPY_Return,US_Corp_Return,TreasInd_Return,Cash_Return
Date,,,,,,,,,,,,,,,,,
2001-02-01,4.908,1118.279500,5.348156,184.4,7.87,4.2,123.949997,4.72,13248.454667,0.188,2.962,0.027871,0.018872,-0.095388,0.014506,0.003573,0.000219
2001-03-01,4.915,1135.408261,4.963977,184.7,7.84,4.3,116.690002,4.18,13274.219333,0.735,2.925,0.026111,0.014697,-0.058572,0.015317,0.001863,0.000044
2001-04-01,5.338,1131.094000,4.613882,185.1,8.07,4.4,126.660004,3.83,13299.984000,1.508,2.732,0.026622,0.010572,0.085440,-0.003800,0.002098,0.000124
2001-05-01,5.413,1137.175217,4.103661,185.3,8.07,4.3,125.949997,3.54,13281.584000,1.873,2.657,0.025457,0.008725,-0.005606,0.005376,0.002226,0.000106
2001-06-01,5.390,1153.920455,3.834078,186.0,7.97,4.5,122.599998,3.56,13263.184000,1.830,2.580,0.027057,0.006880,-0.026598,0.014725,0.001951,0.000228


In [53]:
#Quick function to get num_sd standard deviations away from the median.
#Up pos determines if up is good or bad. Idea being that good scenario is at index 1, bad scenario at index 0.
def get_range(data, variable, num_sd, up_pos = True):
    i = 1 if up_pos else -1
    
    v = data[variable]
    v = v[(v < v.quantile(.75) ) & (v > v.quantile(.25))]
    m = v.mean()
    s = v.std()
    
    print("Variable: {} \t Lower: {:.3f} \t Mean: {:.3f} \t Upper: {:.3f}".format(variable, m-i*num_sd*s, m, m+i*num_sd*s))
    
    return tuple((m - i * num_sd * s, m + i * num_sd * s))

get_range(data, 'UNRATE', 1, False)

Variable: UNRATE 	 Lower: 6.082 	 Mean: 5.561 	 Upper: 5.040


(6.081856487655908, 5.039712139795069)

## Defining Scenarios

- Define a scenario with a boolean string. Make sure to use spaces between operators (for calculation of mahalanobis distances.
- Find empirical mean and covariance matrix of the factors in this scenario.
- Encode scenario as a vector, then find mahalanobis distance using scenario vector and empirical mean, covariance.
- We then convert scenario Mahalanobis distance into likelihood measure:
$$ e^{\frac{-d}{2}}$$
- Rescale probabilities to sum to 1

In [34]:
def get_scenario_vars(scenario):
    #Get the variables in the scenario. Sort alphabetically for consistent replication
    return sorted(list(set([v.split(' ')[0] for v in scenario.split(' & ')])))

def get_scenario_vector(scenario):
    #Get a vector from a scenario.
    vector = {}
    for v in scenario.split(' & '):
        x = v.split(' ')
        cn = x[0]
        val = x[2]
        vector[cn] = float(val)
    vector = pd.DataFrame(vector, index = [0])
    colnames = get_scenario_vars(scenario)
    return vector[colnames].values

In [54]:
#Build the scenarios: Weak is >= 1 sd below median, strong is >= 1 sd above median, normal is in range [m - sd, m + sd].

scen_names = ['Weak', 'Strong', 'Normal']
relevant_vars = ['RGDP_Growth', 'UNRATE', 'CPI_Growth', 'YC_Slope', 'Cred_Spread', 'TNX']
# relevant_vars = ['RGDP_Growth', 'UNRATE', 'CPI_Growth']
# relevant_vars = ['RGDP_Growth']

scenarios = {sn: '' for sn in scen_names}
rgdp_scen = {sn: '' for sn in scen_names}

up_pos = {v:True for v in relevant_vars}
up_pos['UNRATE'] = False
up_pos['YC_Slope'] = False
up_pos['Cred_Spread'] = False

first_run = True

for var in relevant_vars:
    if first_run:
        first_run = False
    else:
        for v in scen_names:
            scenarios[v] += ' & '
    
    low_bound, u_bound = get_range(data, var, 1, up_pos[var])
    
    l_sign = '<=' if up_pos[var] else '>='
    u_sign = '>=' if up_pos[var] else '<='
    
    scenarios['Weak'] += ('{} {} {}'.format(var, l_sign, low_bound))
    scenarios['Normal'] += ('{} {} {} & {} {} {}'.format(var, u_sign, low_bound, var, l_sign, u_bound))
    scenarios['Strong'] += ('{} {} {}'.format(var, u_sign, u_bound))
    
    if var == 'RGDP_Growth':
        rgdp_scen['Weak'] += ('{} {} {}'.format(var, l_sign, low_bound))
        rgdp_scen['Normal'] += ('{} {} {} & {} {} {}'.format(var, u_sign, low_bound, var, l_sign, u_bound))
        rgdp_scen['Strong'] += ('{} {} {}'.format(var, u_sign, u_bound))
    
#     print(var, low_bound, u_bound)
    
scenarios

Variable: RGDP_Growth 	 Lower: 0.018 	 Mean: 0.022 	 Upper: 0.025
Variable: UNRATE 	 Lower: 6.082 	 Mean: 5.561 	 Upper: 5.040
Variable: CPI_Growth 	 Lower: 0.018 	 Mean: 0.020 	 Upper: 0.022
Variable: YC_Slope 	 Lower: 2.407 	 Mean: 1.981 	 Upper: 1.556
Variable: Cred_Spread 	 Lower: 2.886 	 Mean: 2.615 	 Upper: 2.344
Variable: TNX 	 Lower: 2.604 	 Mean: 3.207 	 Upper: 3.811


{'Weak': 'RGDP_Growth <= 0.018263549176152995 & UNRATE >= 6.081856487655908 & CPI_Growth <= 0.018441232660785078 & YC_Slope >= 2.40664577060534 & Cred_Spread >= 2.886327646421964 & TNX <= 2.603999284222915',
 'Strong': 'RGDP_Growth >= 0.02549164641444161 & UNRATE <= 5.039712139795069 & CPI_Growth >= 0.021787474571661598 & YC_Slope <= 1.5562371122775418 & Cred_Spread <= 2.3440146959203774 & TNX >= 3.8105232382996053',
 'Normal': 'RGDP_Growth >= 0.018263549176152995 & RGDP_Growth <= 0.02549164641444161 & UNRATE <= 6.081856487655908 & UNRATE >= 5.039712139795069 & CPI_Growth >= 0.018441232660785078 & CPI_Growth <= 0.021787474571661598 & YC_Slope <= 2.40664577060534 & YC_Slope >= 1.5562371122775418 & Cred_Spread <= 2.886327646421964 & Cred_Spread >= 2.3440146959203774 & TNX >= 2.603999284222915 & TNX <= 3.8105232382996053'}

In [46]:
for theta in [0, 0.5, 1]:

    likelihoods = {}
    l_sum = 0.0

    for scenario_name, scenario in scenarios.items():
        #Get relevant variables for scenario defined above
        scenario_vars = get_scenario_vars(scenario)

        #Get the empirical mean & covariance matrix of scenario vars
        v = (1-theta) * data[scenario_vars].mean().values + theta*data[scenario_vars].tail(1).values

        scen_cov = data[scenario_vars].corr()

        #Encode the scenario as a vector - take empirical averages after conditioning on scenario
        u = data.query(rgdp_scen[scenario_name])[scenario_vars].mean().values
    #     print(u)
        #u = get_scenario_vector(scenario)

        #Mahalanobis distance, converted to likelihood
        l = np.exp(-maha(u, v, scen_cov)/2)
        likelihoods[scenario_name] = l
        l_sum += l

    print('*********** Using Theta = {} ********************'.format(theta))
    probs = {sn: l/l_sum for sn, l in likelihoods.items()}
    print('Likelihoods:\t', likelihoods)
    print('Probs: \t\t', probs)
    
    x = pd.DataFrame(probs, index = [1])
    x.to_csv('data/theta{}_probs.csv'.format(theta))
# print(mahala)

*********** Using Theta = 0 ********************
Likelihoods:	 {'Weak': 0.5531148371105848, 'Strong': 0.6600777652126215, 'Normal': 0.7641983346832434}
Probs: 		 {'Weak': 0.2797195166414282, 'Strong': 0.333812476258188, 'Normal': 0.3864680071003839}
*********** Using Theta = 0.5 ********************
Likelihoods:	 {'Weak': 0.20825439617847846, 'Strong': 0.44973435200135325, 'Normal': 0.46838755576181773}
Probs: 		 {'Weak': 0.18488882929240566, 'Strong': 0.3992754023922108, 'Normal': 0.4158357683153835}
*********** Using Theta = 1 ********************
Likelihoods:	 {'Weak': 0.07560827600497849, 'Strong': 0.1671766856476051, 'Normal': 0.16841573839688878}
Probs: 		 {'Weak': 0.18387195351535615, 'Strong': 0.40655739551876185, 'Normal': 0.40957065096588197}


In [41]:
#Get expected historical returns for each scenario.
#Likely want historical variances also.
returns = pd.DataFrame()
covariances = {}
returns_cols = [x for x in data.columns if 'Return' in x]


for scenario_name, scenario in scenarios.items():
    scen_data = data.query(rgdp_scen[scenario_name])[returns_cols]
    print(scenario_name, scen_data.shape)
    m = scen_data.mean().rename('{}'.format(scenario_name))
    
    m.to_csv('data/{}_ExpReturns.csv'.format(scenario_name))
    
    returns = returns.append(m)
    cov_mat = scen_data.cov()
    cov_mat.to_csv('data/{}_CovMat.csv'.format(scenario_name))
    covariances[scenario_name] = cov_mat
    print(cov_mat)

    
# x
#Column: Scenario
#Row: Expected return
print("\n *********** Returns *********** \n", returns.T)

Weak (78, 4)
                   SPY_Return  US_Corp_Return  TreasInd_Return   Cash_Return
SPY_Return       2.391868e-03    2.581101e-04    -6.950962e-06 -4.762713e-07
US_Corp_Return   2.581101e-04    3.047586e-04     3.663517e-07  3.807103e-07
TreasInd_Return -6.950962e-06    3.663517e-07     5.490431e-06  9.777305e-09
Cash_Return     -4.762713e-07    3.807103e-07     9.777305e-09  9.094241e-09
Strong (79, 4)
                   SPY_Return  US_Corp_Return  TreasInd_Return   Cash_Return
SPY_Return       1.031783e-03    2.893189e-05    -5.718853e-06 -7.711282e-08
US_Corp_Return   2.893189e-05    1.107087e-04     7.947089e-06 -2.403357e-08
TreasInd_Return -5.718853e-06    7.947089e-06     4.287496e-06 -2.235453e-08
Cash_Return     -7.711282e-08   -2.403357e-08    -2.235453e-08  4.821769e-09
Normal (68, 4)
                   SPY_Return  US_Corp_Return  TreasInd_Return   Cash_Return
SPY_Return       1.740639e-03   -3.841915e-05    -2.103896e-05 -3.592690e-07
US_Corp_Return  -3.841915e-05    

In [42]:
for asset_returns in returns_cols:
    r_data = data[relevant_vars]
    r_data = sm.add_constant(r_data)
    endog = data[asset_returns]
    
    model = sm.OLS(endog, r_data)
    res = model.fit(cov_type = 'HC3')
    print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             SPY_Return   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1.987
Date:                Tue, 10 Dec 2019   Prob (F-statistic):             0.0687
Time:                        13:36:11   Log-Likelihood:                 406.39
No. Observations:                 225   AIC:                            -798.8
Df Residuals:                     218   BIC:                            -774.9
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0763      0.038      1.997      

### Disregard below - not right.

In [255]:
#Define scenario, use spaces between operators for ease of parsing.
scenario = 'RGDP_Growth = 0.01 & UNRATE = 6'

#Get relevant variables from the scenario defined above.
scenario_vars = [v.split(' ')[0] for v in scenario.split(' & ')]

#Filter data on the above scenario - not sure if useful or not...
scen_data = data.query(scenario)[scenario_vars]

#Get the empirical mean & correlation matrix of scenario vars
v = data[scenario_vars].mean().values
scen_corr = data[scenario_vars].corr()

#Calculate mahalanobis distance, transform to likelihood measure
data['L_1'] = data[scenario_vars].apply(lambda x: np.exp(-maha(x ,v, scen_corr)/2), raw = True, axis = 1)

#Rescale for likelihood
data['Prob_1'] = data['L_1'].dropna()/data['L_1'].sum()
# data['Prob_1'] = data['Prob_1'].fillna(0)
data.Prob_1

Date
1993-01-01    0.007402
1993-04-01    0.008183
1993-07-01    0.009043
1993-10-01    0.009513
1994-01-01    0.010511
1994-04-01    0.011619
1994-07-01    0.013492
1994-10-01    0.015674
1995-01-01    0.014360
1995-04-01    0.015669
1995-07-01    0.015091
1995-10-01    0.013657
1996-01-01    0.014356
1996-04-01    0.014343
1996-07-01    0.013654
1996-10-01    0.011751
1997-01-01    0.012357
1997-04-01    0.011172
1997-07-01    0.010112
1997-10-01    0.009153
1998-01-01    0.008706
1998-04-01    0.007493
1998-07-01    0.008279
1998-10-01    0.008277
1999-01-01    0.007493
1999-04-01    0.007494
1999-07-01    0.007491
1999-10-01    0.006776
2000-01-01    0.006453
2000-04-01    0.005832
                ...   
2012-04-01    0.004721
2012-07-01    0.004720
2012-10-01    0.005764
2013-01-01    0.005219
2013-04-01    0.006371
2013-07-01    0.007406
2013-10-01    0.007785
2014-01-01    0.010500
2014-04-01    0.012841
2014-07-01    0.012840
2014-10-01    0.015094
2015-01-01    0.015092
2015-0

In [23]:
scenario_1 = 'RGDP_Growth = 0.01 & UNRATE = 6'
scenario_2 = 'RGDP_Growth '

# l_1 = np.exp(-maha(np.array([float(v.split(' ')[2]) for v in scenario_1.split(' & ')]), v, scen_corr))

In [25]:
get_scenario_vars(scenario_1)

['RGDP_Growth', 'UNRATE']

In [ ]:
get_s